The purpose of this code is to quantify the nuclei, the size and number of synuclein inclusions for each image.

Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import gaussian, threshold_otsu, threshold_multiotsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_dilation, disk
import czifile
import skimage.io as io
from skimage import exposure
from cellpose import models
model = models.Cellpose(model_type='cyto')

c:\Users\yh1024\Documents\Yiming Huang\.venv\Lib\site-packages\cellpose\resnet_torch.py:275: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename, 

Define Sub Functions

In [2]:
def show_image(image):
    io.imshow(image, cmap='gray')
    plt.show()


def calculate_average_intensity(image):
    """Calculate the average intensity of the image."""
    return np.mean(image)

def find_inclusion_mask(image):
    """Find the inclusion mask using multi-otsu thresholding."""
    thresholds = threshold_multiotsu(image, classes=3)
    regions = np.digitize(image, bins=thresholds)
    inclusion_mask = (regions == 2).astype(int)
    
    return inclusion_mask

In [3]:
def analyze_image(image_path, basename):
    """
    Analyze an image by its individual cells
    Args:
        image_path: Path to the .czi image file.
        basename: Base name for the file, used in the output DataFrame.

    Returns:
        A DataFrame containing details about each image 
        A second DataFrame containing details about each detected cell
    """
    
    # Load the image and squeeze dimensions
    image = czifile.imread(image_path)
    image_squeezed = np.squeeze(image)
    
    # find image with max average intensity
    max_average_intensity = 0
    max_average_intensity_image = None
    for c in range(image_squeezed.shape[0]):
        green_channel = image_squeezed[c, :, :]
        average_intensity = calculate_average_intensity(green_channel)
        if average_intensity > max_average_intensity:
            max_average_intensity = average_intensity
            max_average_intensity_image = green_channel

    mask = find_inclusion_mask(max_average_intensity_image).astype(bool)
    show_image(mask)
    inverse_mask = np.logical_not(mask)
    # show_image(inverse_mask)
    intensities_without_inclusions = []
    intensities_only_inclusions = []
    intensities_whole_image = []
    for c in range(image_squeezed.shape[0]):
        green_channel = image_squeezed[c, :, :]
        average_intensity_whole_image = calculate_average_intensity(green_channel)
        intensities_whole_image.append(average_intensity_whole_image)
        # show_image(green_channel)
        applied_mask_without_inclusions = green_channel[inverse_mask]

        applied_mask_only_inclusions = green_channel [mask]

        average_intensity_without_inclusions = calculate_average_intensity(applied_mask_without_inclusions)
        intensities_without_inclusions.append(average_intensity_without_inclusions)
        average_intensity_only_inclusions = calculate_average_intensity(applied_mask_only_inclusions)
        intensities_only_inclusions.append(average_intensity_only_inclusions)



    df1 = pd.DataFrame({
        "File_Name:": [basename] * image_squeezed.shape[0],
        "Channel": range(image_squeezed.shape[0]),
        "Average_Intensity_Whole_Image": intensities_whole_image,
        "Average_Intensity_Without_Inclusions": intensities_without_inclusions,
        "Average_Intensity_Only_Inclusions": intensities_only_inclusions

    })
    
    
    return df1


In [4]:
def analyze_all_images(image_folder):
    all_data_1 = []
    for well_image in os.listdir(image_folder):
        
        if well_image.lower().endswith(".czi"):  # Filter for CZI files
            
            print(well_image)
            if "conventional" in well_image.lower():
                continue

            well_image_path = os.path.join(image_folder, well_image)
            well_image_base_name = os.path.basename(well_image)[:-4]
            
            df1 = analyze_image(well_image_path, well_image_base_name)
            all_data_1.append(df1)
            
            print("-" * 200)

    combined_df_1 = pd.concat(all_data_1, ignore_index=True)
    file_name = image_folder.replace("/", "_")
    combined_df_1.to_excel(f"{file_name}_analysis.xlsx", index=False)


Define Main Function

In [5]:
# folders = ['012025_FRET_optimization/new_settings','012725_FRET_optimize/C12_settings','012725_FRET_optimize/Lipidtox_settings']
folders = ['020325_FRET_optimize/C12 Settings','020325_FRET_optimize/Lipidtox_Red Settings']

In [ ]:
for folder in folders:
    analyze_all_images(folder)
